## Imports

In [8]:
##-- install for easy interaction with SEG-Y and Seismic Unix formatted seismic data
!pip3 install segyio

In [9]:
import os
#from google.colab import drive
import sys
import time
import copy
import numpy as np
import matplotlib.pyplot as plt


import segyio
from scipy.signal import convolve
from sklearn.utils import shuffle

In [10]:
import torch
import torch.nn as nn
from torch.autograd import grad
from torch.utils.data import Dataset,DataLoader

device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

In [11]:
!nvidia-smi

In [12]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed);

## Data Generation

* Download & Display the seismic velocity model file

In [13]:
##--- get velocity model data

!wget http://s3.amazonaws.com/open.source.geoscience/open_data/bpvelanal2004/vel_z6.25m_x12.5m_exact.segy.gz
!gunzip vel_z6.25m_x12.5m_exact.segy.gz

In [14]:
velocity_file = segyio.open("vel_z6.25m_x12.5m_exact.segy", ignore_geometry = True)
velocity_model = velocity_file.trace.raw[:].T

In [15]:
plt.imshow(velocity_model, cmap=plt.cm.seismic)
plt.show()

* Coefficients & Boundaries

In [16]:
# size of the domain 
window_x = 500   #-- very very slow with 500
window_y = 500  

# upper bounds
ub = np.array([4, window_x, window_y])

# lower bounds
lb = np.array([0, 0, 0])

# sampling rate
dt = 0.0005
dx = 12.5 
dy = 6.25 

FD_coefficients_X = np.array([[0, 0, 0], [1 / (dx ** 2), -2 / (dx ** 2), 1 / (dx ** 2)], [0, 0, 0]])
FD_coefficients_Y = np.array([[0, 1 / (dy ** 2), 0], [0, -2 / (dy ** 2), 0], [0, 1 / (dy ** 2), 0]])

* Finite Difference Simulation

In [17]:
def optimized_step_ABC(next_, curr, prev, vel):
    value = convolve(curr, FD_coefficients_X, 'same') + convolve(curr, FD_coefficients_Y, 'same')
    np.copyto(next_, 2 * curr - prev + (dt * dt) * (vel * vel) * value)
    
    #bottom
    next_[-1, :] = curr[-2, :] + (vel[-1, :] * dt / dy - 1)/(vel[-1, :] * dt / dy + 1) * (next_[-2, :] - curr[-1, :])
    #right
    next_[:, -1] = curr[:, -2] + (vel[:, -1] * dt / dx - 1)/(vel[:, -1] * dt / dx + 1) * (next_[:, -2] - curr[:, -1])
    #left
    next_[:, 0] = curr[:, 1] + (vel[:, 0] * dt / dx - 1)/(vel[:, 0] * dt / dx + 1)* (next_[:, 1] - curr[:, 0])

* Inject Ricker source

In [18]:
def inject_ricker(pressure, ricker_position_x, ricker_position_y, freq, time_step, dt, cutoff_timestep):
    temp = (np.pi ** 2) * (freq ** 2) * ((((time_step - 1) * dt) - 1 / freq) ** 2)
    ricker = (2 * temp - 1) * np.exp(-temp)
    pressure[ricker_position_y][ricker_position_x] += ricker

* Get a Window from data

In [19]:
def get_window(vel, width_x, width_y):
    maximum_x = vel.shape[1]
    maximum_y = vel.shape[0]

    start_x = np.random.randint(0, maximum_x - width_x)
    start_y = np.random.randint(0, maximum_y - width_y)

    return vel[start_y: start_y + width_y, start_x:start_x + width_x]

In [20]:
window_velocity = get_window(velocity_model, window_x, window_y)
maximum_velocity = window_velocity.max()

plt.imshow(window_velocity, cmap=plt.cm.seismic)
plt.show()

* Initialize Simulation Variables

In [21]:
source_frequency = 20
cutoff_timestep = int((2 / source_frequency) / dt)
initial_time = int(cutoff_timestep * 1.5)

print(f"dt = {dt:6f}, dx = {dx:6f}, dy = {dy:6f}")
print(f"cutoff timestep = {cutoff_timestep:5d}")
print(f"initial time steps = {initial_time:5d}")

x_initial = np.arange(lb[1], ub[1])
y_initial = np.arange(lb[2], ub[2])
t_initial = np.arange(lb[0], dt * initial_time, dt)

ricker_position_x = int(x_initial.shape[0] / 2 + 1)
ricker_position_y = int(y_initial.shape[0] / 2 + 1)

pressure = np.zeros((t_initial.shape[0], x_initial.shape[0], y_initial.shape[0]))
print(pressure.shape)

In [22]:
def Simulate_FD():
    for timestep in range(initial_time - 1):
        timestep + 1
        
        if(timestep < cutoff_timestep):
            inject_ricker(pressure[timestep], ricker_position_x, ricker_position_y, source_frequency, timestep, dt, cutoff_timestep)
        
        optimized_step_ABC(pressure[timestep + 1], pressure[timestep], pressure[timestep - 1], window_velocity)
            
        if timestep%50 == 0:
            print(timestep)
Simulate_FD()

In [23]:
plt.figure(figsize=(20,10))
plt.subplot(2,4,1)
plt.imshow(pressure[int(initial_time / 4)], cmap='gray')
plt.subplot(2,4,2)
plt.imshow(pressure[int(initial_time / 2)], cmap='gray')
plt.subplot(2,4,3)
plt.imshow(pressure[int(initial_time * 3 / 4)], cmap='gray')
plt.subplot(2,4,4)
plt.imshow(pressure[initial_time - 1], cmap='gray')
plt.subplot(2,4,5)
plt.plot(pressure[int(initial_time / 4), ricker_position_x])
plt.subplot(2,4,6)
plt.plot(pressure[int(initial_time / 2), ricker_position_x])
plt.subplot(2,4,7)
plt.plot(pressure[int(initial_time * 3 / 4), ricker_position_x])
plt.subplot(2,4,8)
plt.plot(pressure[initial_time -1, ricker_position_x])
plt.show()

In [24]:
def simulate_pressure(Sx=0, Sy=0, verbose = 0):
    '''
    Sx,Sy : ricker source position
    verbose : print details. 0 for no details.
    '''
    pressure_tensor = np.zeros((t_initial.shape[0], x_initial.shape[0], y_initial.shape[0]), dtype=np.float32)
    
    ricker_position_x = Sx
    ricker_position_y = Sy 
    
    for timestep in range(int(initial_time) - 1):
        timestep + 1

        # inject ricker wavelet
        if(timestep < cutoff_timestep):
            inject_ricker(pressure_tensor[timestep], 
                        ricker_position_x, 
                        ricker_position_y, 
                        source_frequency, 
                        timestep, 
                        dt, 
                        cutoff_timestep)
        
        # proceed in time for 1 step
        optimized_step_ABC(pressure_tensor[timestep + 1], 
        pressure_tensor[timestep], 
        pressure_tensor[timestep - 1], 
        window_velocity)
        
        
        if verbose > 0 : #print results for verification only
            r = initial_time // (verbose)
            if (timestep+1) % r   == 0:
                plt.imshow(pressure_tensor[timestep])
                plt.title(f'Source X : {Sx}, Y : {Sy}, Timestep : {timestep}')
                plt.show()
    return pressure_tensor, Sx, Sy

In [25]:
def generate_9_sources_grid(min_x=0, max_x=500, min_y=0, max_y=500):
    '''
    Generates 9 sources seperated by an equal distance.
    '''
    shift = max_x // 4 
    
    min_x += shift
    max_x -= shift
    min_y += shift
    max_y -= shift
    s_list = []
    s = max_x/3
    for i in range(3):
        for j in range(3):
            s_list.append(((s*i)+min_x, (s*j)+min_y))
    
    return s_list

In [26]:
sources_grid = generate_9_sources_grid(max_x=window_x,max_y = window_x)
sources_grid

In [27]:
def generate_dataset(n_of_simulations = 9, time_steps = 5, space_steps=1, verbose=0, r_init = 0):
    '''
    Args:
        n_of_simulations : number of simulations to be concatenated in the output tensor
        time_steps, space_steps : not the entire time dimension, spatial dimension is taken, we skip it by a factor.
        min_x, max_x : the position of the ricker source is initialized randomly in the space of x and y limited by min_d and max_d
        verbose : print details. 0 for no details.

    Returns:
        X_train ,Y_train ,stacked_pressure_tensor
    '''
    
    X_train = None
    Y_train = None
    
    tr, xr, yr = t_initial.shape[0], x_initial.shape[0], y_initial.shape[0] #simulation space range.
    sx,sy = 0,0 #initially zeros
    T, X, Y, SX, SY = np.meshgrid(np.arange(0,tr,time_steps), np.arange(r_init,xr,space_steps), np.arange(r_init,yr,space_steps),sx,sy)
    x_sim = np.vstack((T.flatten(), X.flatten(), Y.flatten(),SX.flatten(),SY.flatten())).T
    
    stacked_pressure_tensor = {}

    for fd_sim in range(n_of_simulations):
        
        #--- Get Unique Source position
#         while(1):
#             (sx,sy) = generate_random_source(min_x,max_x,min_x,max_x)
#             if (sx,sy) not in stacked_pressure_tensor.keys():
#                 stacked_pressure_tensor[(sx,sy)] = 0
#                 break
        
        (sx, sy) = sources_grid[fd_sim]
        stacked_pressure_tensor[(sx,sy)] = 0
        
        print(f'-- Source #{fd_sim+1} -- Sx={sx}, Sy={sy}')

        #--- Generate Simulation Data
        p_tensor,_,_ = simulate_pressure(Sx = int(sx), Sy = int(sy) , verbose = verbose) 
        stacked_pressure_tensor[(sx,sy)] = p_tensor

        #update_source
        x_sim[:,3] = sx 
        x_sim[:,4] = sy
        
        #--- initialize X_train or append new simulation to it.
        if X_train is None:
            X_train = x_sim.copy()
            y_sim   = np.zeros( (len(X_train),1) )
        else:
            X_train = np.vstack((X_train,x_sim))
        
        #pressure labels initialized once 
        # ~ need optimization takes most of the time
        for i in range(len(x_sim)): 
            t,x,y,_,_ = x_sim[i]
            y_sim[i] = p_tensor[t, x, y]
            #y_sim[i] = p_tensor[tuple(x_sim[i,[0,1,2]])] #same as above
                                      
        if Y_train is None:
            Y_train = y_sim.copy() 
        else:
            Y_train = np.vstack((Y_train,y_sim)) 
        
    return X_train, Y_train ,stacked_pressure_tensor

In [47]:
X_train, Y_train, stacked_pressure_tensor = generate_dataset(n_of_simulations = 9, time_steps = 20, space_steps = 3, verbose=3,r_init=2)

In [29]:
print(X_train.shape)
print(stacked_pressure_tensor.keys())

In [30]:
#-- converting to tensor
X_train_tensor = torch.tensor(X_train,dtype=torch.float32) 
X_train_tensor = X_train_tensor.to(device)

Y_train_tensor = torch.tensor(Y_train,dtype=torch.float32) 
Y_train_tensor = Y_train_tensor.to(device)

In [31]:
class PressureDataSet(Dataset):
    def __init__(self):
        self.n_samples = X_train_tensor.size()[0]

    def __getitem__(self, index):
        t,x,y,sx,sy = X_train_tensor[index]
        pressure = Y_train_tensor[index]
        
        return t,x,y,sx,sy, pressure
    
    def __len__(self):
        return self.n_samples

In [32]:
pressure_dataset = PressureDataSet()

pressure_dataloader = DataLoader(dataset = pressure_dataset, batch_size = 1024 * 2, shuffle=True)

n_of_batches = pressure_dataloader.dataset.n_samples / pressure_dataloader.batch_size
print(n_of_batches)

In [ ]:
#-- check dataloader
for t,x,y,sx,sy,pressure in pressure_dataloader:
    #print(t,x,y,sx,sy,pressure)
    #print(pressure[0])
    break

## Model 

In [35]:
class PINN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(PINN,self).__init__()
        self.tanh = nn.Tanh()
        self.linear1 = nn.Linear(input_size,hidden_size)
        self.linear2 = nn.Linear(hidden_size,hidden_size)
        self.linear3 = nn.Linear(hidden_size,hidden_size)
        self.linear4 = nn.Linear(hidden_size,hidden_size)
        self.linear5 = nn.Linear(hidden_size,hidden_size)
        self.linear6 = nn.Linear(hidden_size,hidden_size)
        self.linear7 = nn.Linear(hidden_size,hidden_size)
        self.linear8 = nn.Linear(hidden_size,hidden_size)
        self.linear9 = nn.Linear(hidden_size,hidden_size)
        self.linear10 = nn.Linear(hidden_size,hidden_size)
        self.linearoutput = nn.Linear(hidden_size,output_size)
        
    def forward(self,t,x,y,sx,sy):
        u = torch.stack((t, x, y, sx, sy), 1)
        out = self.tanh(self.linear1(u))
        
        out = self.tanh(self.linear2(out))
        out = self.tanh(self.linear3(out))
        out = self.tanh(self.linear4(out))
        out = self.tanh(self.linear5(out))
        out = self.tanh(self.linear6(out))
        out = self.tanh(self.linear7(out))
        out = self.tanh(self.linear8(out))

        out = self.linearoutput(out)
        return out

# pinn_model = PINN(5,1024,1)
# pinn_model = pinn_model.to(device)

In [ ]:
#-- track best model at lowest loss
best_model = 0
lowest_loss = 10

## Loading Model from drive

In [ ]:
# !conda install -y gdown
# !gdown https://drive.google.com/uc?id=1tJ82Hg-Gaxvo4aaX2vmKFXAcmmi9mnxu

In [ ]:
#Load
# PATH = 'pinn_model_50epoch.model'
# pinn_model = torch.load(PATH,map_location=torch.device(device))
# pinn_model.to(device)

In [37]:
##last
PATH = '../input/pinn-segy-1812-400/pinn_model_1e-05_1639873461.pth'
pinn_model = torch.load(PATH,map_location=torch.device(device))
pinn_model.to(device)

In [ ]:
#just check
for t,x,y,sx,sy,pressure in pressure_dataloader:
    print(pinn_model(t,x,y,sx,sy)[0])
    break

## Visualization Functions 

* depends on window_size

In [38]:
#-- just to loop on the required time frame in one instruction.
X_ ,Y_ = torch.meshgrid(torch.arange(0,window_x,dtype=torch.float32),torch.arange(0,window_x,dtype=torch.float32)) 
X_ = X_.flatten().to(device)
Y_ = Y_.flatten().to(device)

In [39]:
cm = 'gray'
def predict_pressure_structured(model, time_, stacked_pressure_tensor, Sx, Sy):
    xy_range = window_x
    T_  = torch.full_like(X_,time_).to(device)
    Sx_ = torch.full_like(X_,Sx).to(device)
    Sy_ = torch.full_like(X_,Sy).to(device)
    
    model.eval()
    with torch.no_grad():
        pressure_t = model(T_,X_,Y_,Sx_,Sy_).view(xy_range,xy_range)
 
        fig = plt.figure(figsize=(9,9))
        
        plt.subplot(1,2,1)
        plt.imshow(pressure_t.cpu().detach().numpy(), cmap=cm) 
        plt.title('NN Prediction')

    if((Sx,Sy) in stacked_pressure_tensor.keys()):
        if time_ < stacked_pressure_tensor[(Sx,Sy)].shape[0]:
            plt.subplot(1,2,2)
            plt.imshow(stacked_pressure_tensor[(Sx,Sy)][time_], cmap=cm)
            plt.title('FD Simulation')

    plt.tight_layout()
    plt.suptitle(f'-- Time_Idx : {time_}, Sx : {round(Sx,2)}, Sy : {round(Sy,2)} --',y=0.8,size=25)
    plt.show()
    model.train()

---

## Training with Neural Network loss

---

In [40]:
criterion = nn.MSELoss()
opt_Adam = torch.optim.Adam(pinn_model.parameters(),lr=5e-6)

In [42]:
max_epochs = 0

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt_Adam ,T_max= (max_epochs//2) + 1 ,eta_min=1e-7)
lst_keys = list(stacked_pressure_tensor.keys())
#-- track model loss
call_loss = lambda x : x < 1e-25
self_train = True

pinn_model.train()
for epoch in range(max_epochs):
    print(f'------- epoch {epoch+1} --------')
    start_time = time.time()
    
    for batch_idx,(t,x,y,sx,sy,p) in enumerate(pressure_dataloader):

        #-- forward
        y_nn = pinn_model(t,x,y,sx,sy)
        
        loss1 = criterion(p.reshape(-1,1), y_nn)
        
        if(call_loss(loss1.item())):
            print(f'\nAborting Training, loss : {loss1.item()}')
            self_train = False
            break
        
        #-- backprop loss
        opt_Adam.zero_grad()
        loss1.backward()
        opt_Adam.step()
        
        if(batch_idx % (n_of_batches//5) == 0 ):
            print(f'{epoch}-batch : {batch_idx} / {n_of_batches}, loss1 : {loss1.item()}')
            if loss1.item() < lowest_loss:
                lowest_loss = loss1.item()
                best_model = copy.deepcopy(pinn_model)
                print('----- best model, epoch : ',epoch,' ,loss',lowest_loss)
    
    #-- update learning rate
    lr_scheduler.step()
    lr = opt_Adam.param_groups[0]['lr'] 
    
    
    #-- print epoch details
    print(f'> lr : {lr}')
    end_time = time.time()
    print(f'> epoch {epoch+1}, elapsed : {end_time-start_time} seconds')
    if(not self_train):
        break
    
    #-- vizualise inference
    rsrc = np.random.choice(range(len(lst_keys)),size=1)[0]
    r_src_x, r_src_y  = lst_keys[rsrc]
    
    #predict_pressure_structured(pinn_model,150,stacked_pressure_tensor,r_src_x, r_src_y)
    predict_pressure_structured(pinn_model,230,stacked_pressure_tensor,r_src_x, r_src_y)
#print('Lowest Loss : ', lowest_loss)

#### Save Model

In [ ]:
# torch.save(pinn_model,f'pinn_model_{round(lowest_loss,5)}_{int(time.time())}.pth')
# torch.save(best_model,f'best_model_{round(lowest_loss,5)}_{int(time.time())}.pth')

In [ ]:
# drive.mount('/content/gdrive')
# PATH = f'/content/gdrive/MyDrive/AI/Projects/Physics Modeling NN/pinn_model_5epch_segy{lowest_loss}.pth' 
# torch.save(pinn_model,PATH)

## Predictions 

In [ ]:
# for (Sx,Sy) in stacked_pressure_tensor.keys():
#     #predict_pressure_structured(50,stacked_pressure_tensor,Sx,Sy)
#     #predict_pressure_structured(90,stacked_pressure_tensor,Sx,Sy)
#     #predict_pressure_structured(110,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(pinn_model,200,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(pinn_model,230,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(pinn_model,280,stacked_pressure_tensor,Sx,Sy)
#     #predict_pressure_structured(pinn_model,290,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(pinn_model,310,stacked_pressure_tensor,Sx,Sy)

## Best model

In [ ]:
# for (Sx,Sy) in stacked_pressure_tensor.keys():
#     #predict_pressure_structured(50,stacked_pressure_tensor,Sx,Sy)
#     #predict_pressure_structured(90,stacked_pressure_tensor,Sx,Sy)
#     #predict_pressure_structured(110,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(best_model,200,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(best_model,230,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(best_model,280,stacked_pressure_tensor,Sx,Sy)
#     #predict_pressure_structured(best_model,290,stacked_pressure_tensor,Sx,Sy)
#     predict_pressure_structured(best_model,310,stacked_pressure_tensor,Sx,Sy)

## Physics Loss

### Gradient Calculations

In [43]:
def calculate_grad(f,wrt):
    grads = grad(f, wrt,grad_outputs = torch.ones_like(f),retain_graph=True,allow_unused=False,create_graph=True)[0]
    return grads

In [44]:
max_velocity_sq = maximum_velocity ** 2
def physics_loss(u,t,x,y):
    u_t = calculate_grad(u,t)
    u_x = calculate_grad(u,x)
    u_y = calculate_grad(u,y)
 
    u_tt = calculate_grad(u_t,t)
    u_xx = calculate_grad(u_x,x)
    u_yy = calculate_grad(u_y,y)
    return u_xx + u_yy - (u_tt / max_velocity_sq) # solving => d2u/dx2 + d2u/dy2 - (1/v^2)d2u/dt2 = 0

In [49]:
criterion = nn.MSELoss()
opt_Adam = torch.optim.Adam(pinn_model.parameters(),lr=1e-6)

In [50]:
max_epochs = 2

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt_Adam ,T_max= (max_epochs//2)+1,eta_min=1e-7) #restart lr half trastacked_pressure_tensor loop
call_loss = lambda x : x < 1e-15 
lowest_loss = 10
self_train = True
lst_keys = list(stacked_pressure_tensor.keys()) 

#escape_rate = 0.9
pinn_model.train()
for epoch in range(max_epochs):
    print(f'------- epoch {epoch+1} --------')
    start_time = time.time()
    
    for batch_idx,(t,x,y,sx,sy,p) in enumerate(pressure_dataloader):
        
        #--- to acc training 
        #if np.random.rand() < escape_rate:
            #continue 
            
        t.requires_grad=True;x.requires_grad=True;y.requires_grad=True
        
        ##--- forward prop + loss calculation (NN & Physics)
        y_nn = pinn_model(t,x,y,sx,sy)
        
        loss1 = criterion(p.reshape(-1,1), y_nn)
        l_physics = torch.mean(torch.abs(physics_loss(y_nn,t,x,y)))
        loss_total = loss1 + l_physics

        
        ##--- desired loss callback
        if(call_loss(loss_total.item())):
            print(f'\nAborting Training, loss : {loss_total.item()}')
            self_train = False
            break
        
        
        ##--- backprop
        pinn_model.zero_grad()
        loss_total.backward(retain_graph=True)
        opt_Adam.step()
        
        ##--- print batch progress
        if(batch_idx % (n_of_batches//2) == 0 ):
            if(l_physics != None):
                print(f'batch : {batch_idx} / {n_of_batches}, loss_total : {loss_total.item()}, physics_loss : {l_physics.item()}, NN_loss : {loss1.item()}')
            else:
                print(f'batch : {batch_idx} / {n_of_batches}, loss_total : {loss_total.item()}')
            if loss_total.item() < lowest_loss:
                lowest_loss = loss_total.item()
                
                
    #-- update learning rate
    lr_scheduler.step()
    lr = opt_Adam.param_groups[0]['lr'] 
    
    
    #-- print epoch details
    print(f'> lr : {lr}')
    end_time = time.time()
    print(f'> epoch {epoch+1}, elapsed : {end_time-start_time} seconds')
    print(f'loss_total : {loss_total.item()}, physics_loss : {l_physics.item()}, NN_loss : {loss1.item()}')

    
    #-- vizualise inference
    rsrc = np.random.choice(range(len(lst_keys)),size=1)[0]
    r_src_x, r_src_y  = lst_keys[rsrc]
    predict_pressure_structured(pinn_model,90,stacked_pressure_tensor,r_src_x, r_src_y)
    predict_pressure_structured(pinn_model,130,stacked_pressure_tensor,r_src_x, r_src_y)
        
    
    
    #-- check for exit
    if(not self_train):
        break
        
print('Lowest Loss : ',lowest_loss)

## Predict with Physics loss added

In [51]:
for (Sx,Sy) in stacked_pressure_tensor.keys():
    predict_pressure_structured(pinn_model,150,stacked_pressure_tensor,Sx,Sy)
    predict_pressure_structured(pinn_model,190,stacked_pressure_tensor,Sx,Sy)
    predict_pressure_structured(pinn_model,210,stacked_pressure_tensor,Sx,Sy)
    predict_pressure_structured(pinn_model,250,stacked_pressure_tensor,Sx,Sy)
    predict_pressure_structured(pinn_model,270,stacked_pressure_tensor,Sx,Sy)

## Save Model

In [ ]:
torch.save(pinn_model,'pinn_model_physics100.model')